In [ ]:
! pip install selenium #驚嘆號相當在終端機下指令

# 需先確認欲使用的瀏覽器driver是否有下載，同位址可直接執行，不同的話需修改路徑

In [7]:
'''匯入套件'''
# 操作browser的API
from selenium import webdriver

# #處理逾時例外的工具        ?
from selenium.common.exceptions import TimeoutException     

#面對動態網頁，等待某個元素出現的工具，通常與exptected_conditions搭配         ?
from selenium.webdriver.support.ui import WebDriverWait

# #搭配WebDriverWait使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行    ?
from selenium.webdriver.support import expected_conditions as EC

#期待元素出現要透過什麼方式指定，通常與EC、WebDriverWait一起使用
from selenium.webdriver.common.by import By

#強制等待(執行期間的休息)
from time import sleep

#整理json使用的工具
import json

#執行commcand的時候用
import os

#啟動瀏覽器工具的選項，用一個參數來作為添加條件的基礎         
start = webdriver.ChromeOptions()
start.add_argument('--start-maximized') #最大化
start.add_argument('--incognito') #無痕
start.add_argument('--disable-popup-blocking')

#使用Chrome的Webdriver開啟瀏覽器視窗，單純這行只是開啟空白瀏覽器，所以一般會讓()裡的options屬性接一個自訂的參數(含有瀏覽器設定的條件)
driver = webdriver.Chrome(options=start)

#放置爬取的資料
listData=[]

#走訪頁面
def visit():
    driver.get('https://www.youtube.com/')

#輸入關鍵字
def search():
    #在搜尋欄輸入名稱，find_element為查找元素
    txtInput = driver.find_element(By.CSS_SELECTOR,'input#search')  #同一種效果查詢id為search  By.ID,'search'，可以By許多東西
    txtInput.send_keys('林俊傑') #send_keys 輸入文字
    #按下送出
    btnInput = driver.find_element(By.CSS_SELECTOR,'button#search-icon-legacy')
    btnInput.click()
    
#滾動頁面，滾到底代表捲動的量超過內部高度
def scroll():
    #瀏覽器內部的高度
    innerHeightOfWindow = 0
    
    #當前捲動的量(相當於高度)
    theHeightRightNow = 0
    
    #在捲動到沒有動態元素產生前，持續捲動
    while theHeightRightNow <= innerHeightOfWindow:
        #每次移動高度
        theHeightRightNow += 300
        
        #加入捲動的js code  IIFE，selenium支援js在python裡
        #為了將theHeightRightNow寫進function的top裡，用'''(可讓字串多行，將此字串塞進js_scroll再用excute_script執行code)搭配format
        js_scroll='''(
            function(){{
                window.scrollTo({{
                    top:{},
                    behavior:'smooth'
                }});
            }}
        )()'''.format(theHeightRightNow) #format不知道塞進哪個{}因此在非目標的{}都多加一組{}，類似跳脫字元的概念
        
        #執行js code
        driver.execute_script(js_scroll)
        
        #強制等待
        sleep(0.5)
        
        #透過執行js code來取得捲動後的當前「內部總高度」
        innerHeightOfWindow = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )
        
        #印出滾動距離
        print(f"innerHeightOfWindow:{innerHeightOfWindow},theHeightRightNow:{theHeightRightNow}")
        
        #為了實驗功能，捲動超過一定的距離，就結束程式
        if theHeightRightNow >=1200:
            break
            
#分析頁面元素資訊
def parse():
    #取得主要元素的集合
    ytd_video_renderers = driver.find_elements(
        By.CSS_SELECTOR,'ytd-video-renderer.style-scope'
    )
    
    #逐一檢視元素
    for element in ytd_video_renderers:
        #印出分隔文字
        print("-"*40)
        
        #取得圖片連結
        img = element.find_element(                #這邊到時把element改成driver看有沒有差
            By.CSS_SELECTOR,'ytd-thumbnail.style-scope.ytd-video-renderer img#img' 
        )                                          #(img#img)為在element ytd-thumbnail..故用空白接        
        imgSrc = img.get_attribute('src') #取得img元素裡的src屬性值 
        print(imgSrc)
        
        #取得資料(影片)名稱
        a = element.find_element(By.CSS_SELECTOR,'a#video-title')
        aTitle = a.text  #取得a元素的innertext即為名稱，另一種方法: aTitle = a.get_attribute('innerText')
        print(aTitle)
        
        #取得youtube連結
        aLin = element.find_element(By.CSS_SELECTOR,'a#thumbnail')  #這裡不取用資料名稱的a而是重新抓一個看可不可行
        aLink = aLin.get_attribute('href')
        print(aLink)
        
        #取得youtube ID
        youtube_id = aLink.split("v=")[1] #split以v做分割，取得的是陣列，因此可以用[]來取值  [0]= /watch?  v=   [1]= HDE6hmihdZk  
        print(youtube_id)
        
        #放資料到list當中，將資料以dict形式塞進list
        listData.append({
                "id":youtube_id,
                "title":aTitle,
                "link":aLink,
                "img":imgSrc
        })
        
#將list存成json
def saveJson():
    fp = open("youtube.json",'w',encoding='utf-8')
    fp.write(json.dumps(listData,ensure_ascii=False)) #dumps就是stringify 如果dict裡有中文，需用ensure_ascii=False，
    fp.close() #有開就一定要有關                       #以確保中文不會被ascii取代，因為dumps預設是用ascii解析

#關閉瀏覽器
def close():
    driver.quit()

#下載檔案
def download():
    #開啟json檔案
    fp = open('youtube.json','r',encoding='utf-8')
    
    #取得json字串
    strJson = fp.read()
    
    #關閉檔案
    fp.close()
    
    #將json轉成list(裡面是dict集合)
    listresult = json.loads(strJson) #loads=parse
    
    #由終端機下指令下載檔案
    for index,obj in enumerate(listresult):#利用enumerate可同時取出該值索引及該值
        if index==4:
            os.system("youtube-dl.exe -f mp4 -i {} -o {}".format(obj['link'], "%(id)s.%(ext)s"))#相當於在此檔案的資料夾開啟cmd，youtube-dl.exe也在同一路徑下
        #-f -o -i為youtube-dl的指令 -f為設定格式(但有%(ext)s後好像有沒有他都沒差) -o為設定檔案名稱 -i為ignore errors
        #'%(id)s.%(ext)s'   以這為基礎用-o命名(id為video identifier ext則為檔案的副檔名)  這裡的id和可以帶入的title是指哪邊的id和title
            
#選取搜尋條件  讓他選擇4分鐘內的影片再捲動，避免網頁load過久出現錯誤，故寫try except處理
def filterFun():
    try:
        #等待篩選元素出現
        WebDriverWait(driver,10).until(  #等待10秒，若10秒內條件達成則繼續執行後續code，若超過10秒仍無，則拋出異常處理
            EC.presence_of_element_located(
                (By.CSS_SELECTOR,"tp-yt-paper-button#button.style-text")
            )
        )
        driver.find_element(By.CSS_SELECTOR,'tp-yt-paper-button#button.style-text').click()
        sleep(2)
        driver.find_element(By.LINK_TEXT, '4 分鐘內').click()
        sleep(4)
    except TimeoutException:
        print("等待逾時，即將關閉瀏覽器")
        sleep(3)
        driver.quit()
#主程式
if __name__=="__main__":
    visit()
    search()
    filterFun()
    scroll()
    parse()
    saveJson()
    close()
    download()

innerHeightOfWindow:4561,theHeightRightNow:300
innerHeightOfWindow:4561,theHeightRightNow:600
innerHeightOfWindow:4561,theHeightRightNow:900
innerHeightOfWindow:4561,theHeightRightNow:1200
----------------------------------------
https://i.ytimg.com/vi/LPSFjYh7TTw/hq720.jpg?sqp=-oaymwEcCNAFEJQDSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLCc4grV8kChgoWEg2WpyXKZ6lVHEQ
JJ Lin 林俊傑 ft. Anne-Marie 《Bedroom》Official Lyric Video
https://www.youtube.com/watch?v=LPSFjYh7TTw
LPSFjYh7TTw
----------------------------------------
https://i.ytimg.com/vi/IJ5M4cDkvi8/hq720.jpg?sqp=-oaymwEcCNAFEJQDSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLDCHl9OXhv9aCBVFLAFtGZYcnpmUg
五月天阿信點歌《我不願讓你一個人》林俊傑暖唱《飛鳥和蟬》好聽哭～
https://www.youtube.com/watch?v=IJ5M4cDkvi8
IJ5M4cDkvi8
----------------------------------------
https://i.ytimg.com/vi/YqLwJZoV8DY/hq720.jpg?sqp=-oaymwEcCNAFEJQDSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLCJt_q5xPmVZfQRhNXTPHvbTIkslg
[avex官方HD] 王嘉爾 Jackson Wang & 林俊傑 JJ Lin - 過 Should've Let Go 官方完整版MV
https://w